<a href="https://colab.research.google.com/github/ykalathiya-2/unsloath/blob/main/unsloath_full_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm synthetic-data-kit==0.0.3
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [22]:
from unsloth.dataprep import SyntheticDataKit

torch_dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float32
generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

INFO 11-10 03:48:27 [vllm_utils.py:700] Unsloth: Patching vLLM v1 graph capture
INFO 11-10 03:48:27 [vllm_utils.py:730] Unsloth: Patching vLLM v0 graph capture
Unsloth: Using dtype = torch.float16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 78.21%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.55 GB. Also swap space = 4 GB.
Unsloth: `cudagraph_mode` is not in `from vllm.config import CompilationConfig`
Unsloth: Disabling `disable_cascade_attn` in vLLM to allow for better on policy RL!
vLLM STDOUT: INFO 11-10 03:48:39 [__init__.py:244] Automatically detected platform cuda.
vLLM STDOUT: INFO 11-10 03:48:41 [api_server.py:1395] vLLM API server version 0.9.2
vLLM STDOUT: INFO 11-10 03:48:41 [cli_args.py:325] non-default args: {'model': 'unsloth/Llama-3.2-3B-Instruct', 'dtype': 'float16', 

## Generate QA Pairs + Auto clean data
We now use synthetic data kit for question answer pair generation:

In [3]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

Check if it succeeded:

In [4]:
!synthetic-data-kit system-check

vLLM STDOUT: INFO:     127.0.0.1:59156 - "GET /v1/models HTTP/1.1" 200 OK
 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1762745765, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-65276389bc76471697bf6e384f4faa19', 'object': 'model_permission', 
'created': 1762745765, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


## Document Parsing (PDF, CSV, HTML etc.)
Now, let's take the Byte Latent Transformer: Patches Scale Better Than Tokens research paper found at https://arxiv.org/abs/2412.09871 and covert it to Q&A pairs in order to finetune Llama 3.2!

In [5]:
# Byte Latent Transformer: Patches Scale Better Than Tokens paper in HTML format
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://arxiv.org/html/2412.09871v1"

# Truncate document
filenames = generator.chunk_data("data/output/arxiv_org.txt")
print(len(filenames), filenames[:3])

⠏ Processing https://arxiv.org/html/2412.09871v1...
 Text successfully extracted to data/output/arxiv_org.txt
34 ['data/output/arxiv_org_0.txt', 'data/output/arxiv_org_1.txt', 'data/output/arxiv_org_2.txt']


We see around 37 chunks of data. We now call synthetic-data-kit to create some pairs of data for 3 of our chunks.

You can process more chunks, but it'll be much slower!

Using `--num-pairs` will generate **approximately** that many QA pairs. However it might be shorter or longer depending on the `max_seq_length` of the loaded up model. So if you specify 100, you might only get 10 since the model's max sequence length is capped.

In [6]:
import time
# Process 3 chunks for now -> can increase but slower!
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

vLLM STDOUT: INFO:     127.0.0.1:42442 - "GET /v1/models HTTP/1.1" 200 OK
vLLM STDOUT: INFO:     127.0.0.1:42446 - "GET /v1/models HTTP/1.1" 200 OK
⠦ Generating qa content from data/output/arxiv_org_0.txt...vLLM STDOUT: INFO 11-10 03:36:08 [chat_utils.py:444] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
vLLM STDOUT: INFO 11-10 03:36:08 [logger.py:43] Received request chatcmpl-ed88e9df0a32425a90db8933f5d3db45: prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 10 Nov 2025\n\nSummarize this document in 3-5 sentences, focusing on the main topic and key concepts.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nByte Latent Transformer: Patches Scale Better Than Tokens\n1 Introduction\n2 Patching: From Individual Bytes to Groups of Bytes\n2.1 Strided Patching Every K Bytes\n2.2 Space Patching\n2.3 Entropy Patching: Using Next-Byte Entropies 

We now convert the generated datasets into QA formats so we can load it for finetuning:

In [7]:
qa_pairs_filenames = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


Let's load up the data and see what the synthetic data looks like!

In [8]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/arxiv_org_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [9]:
dataset[-1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'How does BLT models differ from MegaByte models in terms of patching?',
   'role': 'user'},
  {'content': 'BLT models group bytes into patches by striding every four bytes, unlike MegaByte models.',
   'role': 'assistant'}]}

Finally free vLLM process to save memory and to allow for finetuning!

In [10]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
vLLM STDOUT: INFO 11-10 03:37:27 [launcher.py:80] Shutting down FastAPI HTTP server.
Server terminated gracefully.


### Fine-tuning Synthetic Dataset with Unsloth

In [39]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "HuggingFaceTB/SmolLM2-135M",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = True, # [NEW!] We have full finetuning now!
    token = userdata.get('HUGGING_FACE_TOKEN'), # use one if using gated models
    torch_dtype=torch.float16,
)

==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.
HuggingFaceTB/SmolLM2-135M does not have a padding token! Will use pad_token = <|endoftext|>.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.2` format for conversation style finetunes. The chat template renders conversations like below: (Cutting Knowledge Date is by default there!)

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 May 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is 1+1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2<|eot_id|>
```

In [54]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

Model does not have a padding token! Will use pad_token = <|endoftext|>.


In [55]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

See result of the first row:

In [56]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the main contribution of the Byte Latent Transformer (BLT) architecture?',
   'role': 'user'},
  {'content': 'The BLT architecture introduces a new byte-level LLM that matches tokenization-based LLM performance at scale with significant improvements in inference efficiency and robustness.',
   'role': 'assistant'}],
 'text': '<|endoftext|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the main contribution of the Byte Latent Transformer (BLT) architecture?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe BLT architecture introduces a new byte-level LLM that matches tokenization-based LLM performance at scale with significant improvements in inference efficiency and robustness.<|eot_id|>'}

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [57]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=12):   0%|          | 0/32 [00:00<?, ? examples/s]

In [58]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.385 GB of memory reserved.


In [59]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32 | Num Epochs = 8 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 134,515,008 of 134,515,008 (100.00% trained)


Step,Training Loss
1,2.386600
2,2.367900
3,2.155000
4,1.747100
5,1.282900
6,1.024600
7,0.829300
8,0.657200
9,0.471400
10,0.487200


In [60]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

28.0194 seconds used for training.
0.47 minutes used for training.
Peak reserved memory = 2.385 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 16.179 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [61]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

28.0194 seconds used for training.
0.47 minutes used for training.
Peak reserved memory = 2.385 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 16.179 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! Use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [ ]:
messages = [
    {"role": "user", "content": "What is the Byte Latent Transformer?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


A novel architecture that uses patches to encode bytes<|eot_id|>


The model learns about the research paper!!

In [ ]:
messages = [
    {"role": "user", "content": "What are some benefits of the BLT?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

Improved inference capabilities, enhanced model robustness, and the ability to scale while maintaining a fixed-inference budget<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [67]:
model.save_pretrained("fine_tuned_llama_3.2")  # Local saving
tokenizer.save_pretrained("fine_tuned_llama_3.2")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('fine_tuned_llama_3.2/tokenizer_config.json',
 'fine_tuned_llama_3.2/special_tokens_map.json',
 'fine_tuned_llama_3.2/chat_template.jinja',
 'fine_tuned_llama_3.2/vocab.json',
 'fine_tuned_llama_3.2/merges.txt',
 'fine_tuned_llama_3.2/added_tokens.json',
 'fine_tuned_llama_3.2/tokenizer.json')